In [6]:
import pandas as pd

In [1]:
import re

# Step 1: Open the file and read its contents
with open('data_sme.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# Step 2: Use a regular expression to split the contents based on any timestamp in the format HH:MM
parts = re.split(r'\d{2}:\d{2}', content)


In [3]:
len(parts)

74794

In [7]:
df = pd.DataFrame(parts, columns=['text'])


In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

11121

In [17]:
df['text'] = df['text'].str.strip()

In [89]:
df.reset_index(drop=True, inplace=True)

In [138]:
df['text'][891]

'Mook\t[Photo]'

In [151]:
PROMPT = "Determine if the following message is likely posted by a seller Think step by step. Respond Start with either YES or NO.\
Message: {}\
Answer format: <CHOICE>[Your Answer]</CHOICE>\
RULES:\
    1. If the message is likely posted by a seller, answer YES.\
        2. If the message is not likely posted by a seller, answer NO.\
            3. If you are unsure, answer NO.\
                4.Answer only YES or NO.\
                    "

In [152]:
from ollama import generate

In [153]:
Answer_list = []

In [161]:
import concurrent.futures

def process_message(start, end):
    for i in range(start, end):
        decide = generate(model='gemma2:2b-instruct-q8_0', prompt=PROMPT.format(df['text'][i]))
        print('Processing message: ', i, 'of', len(df))
        print(decide['response'])
        if 'YES' in decide['response']:
            print('YES')
            Answer_list.append(df['text'][i])

# Determine the chunk size for each thread
chunk_size = len(df) // 3

# Create a ThreadPoolExecutor to manage the threads
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = []
    for i in range(3):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < 2 else len(df)
        futures.append(executor.submit(process_message, start, end))

    # Wait for all threads to complete
    concurrent.futures.wait(futures)



Processing message: Processing message:  3707 of 11121
NO 

 0 of 11121
NO 

Processing message:  7414 of 11121
NO 

Processing message:  3708 of 11121
NO 

Processing message:  1 of 11121
NO 

Processing message:  7415 of 11121
NO 

Processing message:  2 of 11121
NO 

Processing message:  7416 of 11121
NO 

Processing message:  3 of 11121
NO 

Processing message:  7417 of 11121
NO 

Processing message:  4 of 11121
NO 

Processing message:  7418 of 11121
NO 

Processing message:  5 of 11121
NO 

Processing message:  7419 of 11121
NO 

Processing message:  6 of 11121
NO 

Processing message:  3709 of 11121
NO 

**Explanation:**

* **Language:** The language used (Thai) suggests this is likely a customer asking about a specific product/service.  
* **Question Format:** "วันนี้ส้มตำปริญญาเปิดมั้ยคะ" ("Is ssam-tam P-rinya open today?") is a question typical of someone looking for information about availability and services. This isn't common language used by sellers to directly promote th

In [162]:
import re

# Define a function to remove links from a text
def remove_links(text):
    return re.sub(r'http\S+', '', text)

# Apply the function to each item in the Answer_list
Answer_list = [remove_links(text) for text in Answer_list]
Answer_list = [text.split('\t', 1)[-1] for text in Answer_list]

In [163]:
Answer_list

['น.)(emoji)\n(emoji)จัดส่งในวันที่6-7 มิถุนายน2567นะคะ\n(emoji)ส่งทั่ว มทส และระแวกใกล้เคียงค่ะ\n(emoji)ซื้อครบ100 จัดส่งฟรี (มทส)\n(emoji)ซื้อครบ 150 จัดส่งฟรี (ระแวกใกล้เคียง *ไม่ไกลมาก)\n>>>(emoji)เมนูที่เปิดจอง(emoji)>>>\n🍩ทิมเบอร์ริง ไส้ทูเทลล่า ชิ้นละ 25.-\n🍩พอนเกรซซิ่ง🍬 ชิ้นละ 20.-\n🍩พอนช็อคโกแลต🍫 ชิ้นละ20.-\n🍩 Peanut Donuts 🥜 ชิ้นละ20.-\n🍩Popodip เซตละ20.-\n🍩เชลไส้ชาโคล 15.- \n🍩เชลทูโทน 15.-\n🍩ไอซ์ทูโทน 10.-\n🧁Mini Cakes 20.-\n\n(emoji)หมายเหตุ:จองแล้วงดยกเลิกทุกกรณี🙏\n(emoji)\n(emoji)',
 'น้ำเต้าหู้ พร้อมส่งแล้วงับ🔥✨ \n\n👉🏻 น้ำเต้าหู้นมสดร้อน  7.- \n👉🏻น้ำเต้าหู้รสชาเขียวร้อน 10.- หมด🚫\n👉🏻น้ำเต้าหู้รสชาไทยร้อน  10.-\n\nน้ำเต้าหู้เย็น แก้วละ20.- ฟรีเม็ดเจียทุกแก้ว\n👉🏻นมสด\n👉🏻ชาเขียว\n👉🏻ชาไทย\n(ทางร้านแยกน้ำแข็งทุกแก้วค่ะ)\n\n- หวานน้อย หวานปกติ หวานมาก ไม่หวานเลย\n\n🔥พร้อมส่งทุกประตู 🔥\n✨ส่งฟรีตั้งแต่แก้วแรก\n',
 '🍒YouTube premium 🍒\n30 days เมล์ลูกค้า\nไม่ต่อเมล 30฿ \nต่อเมล 60฿\n\n🍒Sticker & Theme 🍒\n50 coins 30฿\n85 coins 50฿\n100 coins 60฿\n150 coins 85฿\n\n🍒Netflix 🍒(เข้าT

In [164]:
df_clean = pd.DataFrame(Answer_list, columns=['text'])

In [165]:
df_clean.to_csv('data_sme_clean.csv', index=False)

In [166]:
len(df_clean)

481